In [113]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import base
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import RidgeCV, LinearRegression, SGDRegressor, Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

# Recommendation Engine

## Problem definition and data format

The goal of a recommendation engine is to match items to users that will prefer them.  Since we have a set of previous ratings, we can use them to make guesses about which items a user will rate highest.  In this tutorial, we will cover two ways of doing this:

1. **Content-based rating:** We can make this guess based only on features of the users or items.
1. **Collaborative Rating:** We could also look at ratings of similar users or similar items.

For an example data set, we are using the [MovieLens 10M dataset](http://files.grouplens.org/datasets/movielens/ml-10m.zip).  This contains about 10M ratings for 10k movies by 72k users.  We will be building applications that attempts to present movies that a given user would rate highly.  The following cell will download and extract the data to the `projects/ml-10M100K/` directory.

In [115]:
%%bash
cd projects
if [ ! -d "ml-10M100K" ]; then
    wget http://files.grouplens.org/datasets/movielens/ml-10m.zip
    unzip ml-10m.zip
    rm ml-10m.zip
fi

bash: line 1: cd: projects: No such file or directory


There are three data files that are unpacked.  `movies.dat` contains movie IDs, titles, and categories.  `tags.dat` contains a list of tag applications; each line representing the application of a single tag to a movie by a user.  Finally, `ratings.dat` contains the ratings of the movies by the users.

Each of these files are in a custom format, with fields separated by `::`.  They're easy enough to parse with Python string processing.

In [116]:
def parse_movie_line(l):
    id_, title, cats = l.strip().split('::')
    return {'id': int(id_), 'title': title, 'year': int(title.rsplit(' ')[-1][1:-1]), 
            'categories': cats.split('|')}

with open('ml-10M100K/movies.dat', 'r') as f:
    df = pd.DataFrame([parse_movie_line(l) for l in f]).set_index('id')

df.head()

,categories,title,year
id,,,
1,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story (1995),1995
2,"[Adventure, Children, Fantasy]",Jumanji (1995),1995
3,"[Comedy, Romance]",Grumpier Old Men (1995),1995
4,"[Comedy, Drama, Romance]",Waiting to Exhale (1995),1995
5,[Comedy],Father of the Bride Part II (1995),1995


In [117]:
def parse_tag_line(l):
    uid, mid, tag, time = l.strip().split('::')
    return {'user_id': int(uid), 'movie_id': int(mid), 'tag': tag}

with open('ml-10M100K/tags.dat', 'r') as f:
    df_tags = pd.DataFrame([parse_tag_line(l) for l in f])

df_tags.head()

,movie_id,tag,user_id
0,4973,excellent!,15
1,1747,politics,20
2,1747,satire,20
3,2424,chick flick 212,20
4,2424,hanks,20


In [118]:
def parse_rating_line(l):
    uid, mid, rating, time = l.strip().split('::')
    return {'user_id': int(uid), 'movie_id': int(mid), 'rating': float(rating)}

with open('ml-10M100K/ratings.dat', 'r') as f:
    df_ratings = pd.DataFrame([parse_rating_line(l) for l in f])

df_ratings.head()

,movie_id,rating,user_id
0,122,5.0,1
1,185,5.0,1
2,231,5.0,1
3,292,5.0,1
4,316,5.0,1


## Feature engineering

We will start with content modeling, based on the movie categories.  If a user rated a bunch of Adventure films highly, a simple scheme would be to recommend more Adventure films.  Of coure, many users may like several genres of movies, so we need a way to represent categories beyond just the string representing the favorite.

The first approach for vectorizing the categories might be to assign numbers to each of the categories.  For example, we could say *Adventure* = 1, *Drama* = 2, *Comedy* = 3, and so on.  This approach, however, signals that there is some order to these genres.  This mapping would suggest that *Drama* = (*Adventure* + *Comedy*)/2, which isn't meaningful.  We don't want a movie that is an *Adventure Comedy* to have the same feature as a *Drama*.

Instead, we need to recognize that genre is a categorical variable, and thus should be encoded with one-hot encoding.  Essentially, this give each genre its own dimension in feature space.  Our *Adventure Comedy* movie will be located along both the *Adventure* and *Comedy* axes, but at 0 along the *Drama* axis.

We will use *Scikit Learn's* transformers to do this conversion.  We don't know the categories *a priori*, as we did with the days of the week, so we will use the `DictVectorizer` to help us.  This transformer takes in a list of dictionaries.  Each key in the dictionaries gets mapped to a column, and the values for those keys are placed in the appropriate column.  Columns for keys that are not present in a particular row are filled with zeros.

In order to use this, we need to transform our list of genres in to a dictionary, with values of one.

In [119]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        def to_dict(l):
            try:
                return {x: 1 for x in l}
            except TypeError:
                return {}
        
        return X[self.col].apply(to_dict)

Handling each transformer manually quickly gets tedious and error-prone. Scikit Learn comes to the rescue with pipelines. Pipelines take a series of transformers and (optionally) an estimator. A pipeline acts as an estimator itself. When it is fit, it fits the first transformer, transforms with the first transformer, uses that value to fit the second transformer, transforms with the second transformer, etc., until finally fitting the estimator. When predict is called on the pipeline, it sends the feature matrix through each of the transformers, before finally calling predict on the estimator.

This pipeline helps us chain transformers together.  Since the pipeline doesn't end in an estimator, it acts as a transformer instead.

In [120]:
cat_pipe = Pipeline([('encoder', DictEncoder('categories')),
                     ('vectorizer', DictVectorizer())])
features = cat_pipe.fit_transform(df)
features

<10681x20 sparse matrix of type '<type 'numpy.float64'>'
	with 21564 stored elements in Compressed Sparse Row format>

The `DictVectorizer` returns a sparse matrix.  This allows efficient operations even on high-dimensional data.

## Nearest Neighbors

Now that we have a way to describe a movie, we want to be able to find other movies that are nearby in feature space.  This is the **nearest neighbors** problem, and Scikit Learn provides a class to handle this.

When the `NearestNeighbors` object is fit, it records all of the rows in such a way that it can efficiently look through them to find which is nearest to one queried later.  The `NearestNeighbors` assumes that the closest points are the ones for which the [Minkowski Distance](https://en.wikipedia.org/wiki/Minkowski_distance) is minimized.  That is, if we have two rows of our feature vector $X_{i\cdot}$ and $X_{j\cdot}$, it minimizes
$$ \|X_{i\cdot} - X_{j\cdot}\|_p^p = \sum_k \left(X_{ik} - X_{jk}\right)^p $$
By default, it computes $p=2$, which is the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance).

![KNN](../images/KnnClassification.svg)

In [121]:
nn = NearestNeighbors(n_neighbors=20).fit(features)

We will discuss how to measure performance of these models later, but for now we will just eyeball the results.  We'll pick out two movies as test cases.  *Toy Story* happens to be the first movie in the data set.  *Dr. Strangelove* is a favorite of the author.

In [122]:
df.iloc[[0, 737]]

,categories,title,year
id,,,
1,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story (1995),1995
750,"[Comedy, War]",Dr. Strangelove or: How I Learned to Stop Worr...,1964


The results for *Toy Story* are reasonable.  All of the movies are animated children's movies, and *Toy Story 2* shows up.  But by relying only on the genres, we are unable to select Pixar movies specifically, for example.

In [123]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

,categories,title,year
id,,,
26662,"[Adventure, Animation, Children, Comedy, Fantasy]",Kiki's Delivery Service (Majo no takkyûbin) (1...,1989
1,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story (1995),1995
47124,"[Adventure, Animation, Children, Comedy, Fantasy]","Ant Bully, The (2006)",2006
45074,"[Adventure, Animation, Children, Comedy, Fantasy]","Wild, The (2006)",2006
53121,"[Adventure, Animation, Children, Comedy, Fantasy]",Shrek the Third (2007),2007
2294,"[Adventure, Animation, Children, Comedy, Fantasy]",Antz (1998),1998
4016,"[Adventure, Animation, Children, Comedy, Fantasy]","Emperor's New Groove, The (2000)",2000
3114,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story 2 (1999),1999
3754,"[Adventure, Animation, Children, Comedy, Fantasy]","Adventures of Rocky and Bullwinkle, The (2000)",2000


This limitation is more apparent when looking a movies similar to *Dr. Strangelove*.  We get all the *War Comedy* movies, but that doesn't mean that they're similar to *Dr. Strangelove*.  While the author has never seen *The Wackiest Ship in the Army*, he is pretty sure it is of a rather different tone.

In [124]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

,categories,title,year
id,,,
1663,"[Comedy, War]",Stripes (1981),1981
1078,"[Comedy, War]",Bananas (1971),1971
63810,"[Comedy, War]",When Willie Comes Marching Home (1950),1950
7292,"[Comedy, War]",Best Defense (1984),1984
750,"[Comedy, War]",Dr. Strangelove or: How I Learned to Stop Worr...,1964
7104,"[Comedy, War]",1941 (1979),1979
1445,"[Comedy, War]",McHale's Navy (1997),1997
6561,"[Comedy, War]","Mouse That Roared, The (1959)",1959
5789,"[Comedy, War]",All the Queen's Men (2001),2001


## Tag Data

The problem is that we're only getting 20 bits of data about each movie from the categories.  We have much more information in the user-applied tags, though, so let's bring that to bear.

As a reminder, the tag data is stored one tag application per row.

In [125]:
df_tags.head()

,movie_id,tag,user_id
0,4973,excellent!,15
1,1747,politics,20
2,1747,satire,20
3,2424,chick flick 212,20
4,2424,hanks,20


We want all tags for each movie, so we group by the movie_id and get a list of tags.

In [126]:
all_tags = df_tags.groupby('movie_id')['tag'].apply(lambda x: x.tolist())
all_tags.head()

movie_id
1    [Pixar, Pixar, Pixar, animation, Pixar, animat...
2    [For children, game, animals, Joe Johnston, Ro...
3    [Funniest Movies, comedinha de velhinhos engra...
4                                         [girl movie]
5    [steve martin, pregnancy, remake, steve martin...
Name: tag, dtype: object

The equivalent of a SQL `join` statement in Pandas is the `merge()` method.  We specify a left join, to keep movies without any tags, and indicate rows should be matched by index, which is the movie_id.

In [127]:
df = df.merge(all_tags.to_frame(), left_index=True, right_index=True, how='left')
df.head()

,categories,title,year,tag
id,,,,
1,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story (1995),1995,"[Pixar, Pixar, Pixar, animation, Pixar, animat..."
2,"[Adventure, Children, Fantasy]",Jumanji (1995),1995,"[For children, game, animals, Joe Johnston, Ro..."
3,"[Comedy, Romance]",Grumpier Old Men (1995),1995,"[Funniest Movies, comedinha de velhinhos engra..."
4,"[Comedy, Drama, Romance]",Waiting to Exhale (1995),1995,[girl movie]
5,[Comedy],Father of the Bride Part II (1995),1995,"[steve martin, pregnancy, remake, steve martin..."


We'll use the same sort of pipeline to one-hot encode the tags.  (It might be better to use an alternative encoding that accounts for the number of times each tag was applied.)  Then, a `FeatureUnion` can join the two one-hot encoded matrices.

In [128]:
tag_pipe = Pipeline([('encoder', DictEncoder('tag')),
                     ('vectorizer', DictVectorizer())])
union = FeatureUnion([('categories', cat_pipe),
                      ('tags', tag_pipe)])

In [129]:
features = union.fit_transform(df)
features

<10681x16549 sparse matrix of type '<type 'numpy.float64'>'
	with 92719 stored elements in Compressed Sparse Row format>

Over 16k features!

In [130]:
nn = NearestNeighbors(n_neighbors=20).fit(features)

With the tags, we now clearly pick out *Toy Story 2* as the most similar to *Toy Story*.  But the rest seem worse.  We're not getting any of the Pixar movies, for instance.

In [131]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

,categories,title,year,tag
id,,,,
1,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story (1995),1995,"[Pixar, Pixar, Pixar, animation, Pixar, animat..."
3114,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story 2 (1999),1999,"[toys, Pixar, Pixar, Disney, Tom Hanks, sequel..."
1064,"[Animation, Children, Comedy]",Aladdin and the King of Thieves (1996),1996,"[Cartoon, Dan Castelanetta, comedy, family]"
5109,"[Adventure, Animation, Children]",Return to Never Land (2002),2002,"[Disney, Disney animated feature, Peter Pan, avi]"
3759,"[Animation, Children, Musical]",Fun and Fancy Free (1947),1947,"[Disney, animation, erlend's DVDs]"
2043,"[Adventure, Children, Fantasy]",Darby O'Gill and the Little People (1959),1959,[Disney]
32153,"[Adventure, Animation, Children, Fantasy]",Once Upon a Forest (1993),1993,NaN
2142,"[Animation, Children, Comedy]","American Tail: Fievel Goes West, An (1991)",1991,[classic]
3945,"[Adventure, Animation, Children]",Digimon: The Movie (2000),2000,[very good]


The problem becomes more clear looking at *Dr. Strangelove*.  *The Mouse that Roared* is a good pick, but most of the rest have no tags.  The problem is that the tag space is so large, and so sparsely populated, that even similar movies will have very few overlapping tags.  As a result, movies with no tags end up being the closest to most movies.

In [132]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

,categories,title,year,tag
id,,,,
750,"[Comedy, War]",Dr. Strangelove or: How I Learned to Stop Worr...,1964,"[Kubrick, classic, nuclear war, Stanley Kubric..."
6561,"[Comedy, War]","Mouse That Roared, The (1959)",1959,"[anti-war, nuclear weapons, satirical]"
5231,[Comedy],Road to Morocco (1942),1942,"[National Film Registry, AFI 100 (Laughs)]"
473,"[Comedy, War]",In the Army Now (1994),1994,NaN
7159,"[Comedy, War]",Two Men Went to War (2003),2003,NaN
5780,[Comedy],Polyester (1981),1981,"[biting, campy, humorous, irreverent, satirical]"
5801,"[Comedy, War]","Russians Are Coming, the Russians Are Coming, ...",1966,NaN
9001,"[Comedy, War]","Wackiest Ship in the Army, The (1960)",1960,NaN
63810,"[Comedy, War]",When Willie Comes Marching Home (1950),1950,NaN


## Dimensional Reduction

We don't actually expect there to be 16k different concepts expressed in the tags.  Rather, several tags may be associated with a single underlying concept.  In the above, we can see tags for *animation* and *cartoon*, for example.

![Compression](../images/map_compression.png)

There are several methods of **dimensional reduction** that attempt to pull out a lower-dimensional structure.  We'll be using Truncated Singular Value Decomposition (SVD), which works well with sparse matrices.  (Other methods include PCA, NMF, etc.)  SVD attempts to find orthogonal directions within feature space, along which the feature matrix has the most variation.  Recall that for $X$, an $n \times p$ feature matrix, we have the [Singular Value Decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition) with

$$ X = U \Sigma V^T $$

where $U$ is a unitary $n \times n$ matrix, $\Sigma$ is a diagonal $n \times p$ matrix, and $V$ is a unitary $p \times p$ matrix.  With this decomposition, we can then truncate the diagonal matrix $\Sigma$ to include only the $m$ dimensions with the most variation.

Our choice of $m=100$ seems reasonable, but can be tuned to adjust the sensitivity to the tags.

![SVD](../images/GaussianScatterPCA.svg)

In [133]:
tag_pipe = Pipeline([('encoder', DictEncoder('tag')),
                     ('vectorizer', DictVectorizer()),
                     ('svd', TruncatedSVD(n_components=100))])
union = FeatureUnion([('categories', cat_pipe),
                      ('tags', tag_pipe)])

In [134]:
features = union.fit_transform(df)
features

<10681x120 sparse matrix of type '<type 'numpy.float64'>'
	with 783464 stored elements in Compressed Sparse Row format>

In [135]:
nn = NearestNeighbors(n_neighbors=20).fit(features)
nn.fit(features)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=20, p=2, radius=1.0)

Now we seem to be getting mostly Disney movies for *Toy Story*, which seems reasonable.

In [136]:
dists, indices = nn.kneighbors(features[0])
df.iloc[indices[0]]

,categories,title,year,tag
id,,,,
1,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story (1995),1995,"[Pixar, Pixar, Pixar, animation, Pixar, animat..."
3114,"[Adventure, Animation, Children, Comedy, Fantasy]",Toy Story 2 (1999),1999,"[toys, Pixar, Pixar, Disney, Tom Hanks, sequel..."
2355,"[Adventure, Animation, Children, Comedy]","Bug's Life, A (1998)",1998,"[talking animals, kid flick, Pixar, Pixar, Pix..."
595,"[Animation, Children, Fantasy, Musical, Romance]",Beauty and the Beast (1991),1991,"[fairy tale, Disney, Disney, Disney, animated ..."
4016,"[Adventure, Animation, Children, Comedy, Fantasy]","Emperor's New Groove, The (2000)",2000,"[Disney, G, okay, Disney, funny, Disney, almos..."
5444,"[Animation, Children]",Lilo & Stitch (2002),2002,"[Disney, Disney, Cute!, DIVX, redemption, Bear..."
1907,"[Adventure, Animation, Children, Comedy, Drama]",Mulan (1998),1998,"[2, fabulous, walt disney, Disney, disney was ..."
45517,"[Animation, Children, Comedy, Fantasy]",Cars (2006),2006,"[Pixar, actually funny, instills good moral va..."
2080,"[Animation, Children, Comedy, Musical, Romance]",Lady and the Tramp (1955),1955,"[children, dogs, romantic, Animation, Children..."


Now, the algorithm finds movies like *The Great Dictator* and *MASH*, which are fairly similar in message to *Dr. Strangelove*.  It also finds *Full Metal Jacket*, another Stanley Kubrick war movie, albeit with a very different tone.

In [137]:
dists, indices = nn.kneighbors(features[737])
df.iloc[indices[0]]

,categories,title,year,tag
id,,,,
750,"[Comedy, War]",Dr. Strangelove or: How I Learned to Stop Worr...,1964,"[Kubrick, classic, nuclear war, Stanley Kubric..."
1247,"[Comedy, Drama, Romance]","Graduate, The (1967)",1967,"[Simon & Garfunkel soundtrack, Simon and Garfu..."
1394,[Comedy],Raising Arizona (1987),1987,"[Arizona, coen bros, Cult classic, Coen Brothe..."
910,"[Comedy, Crime]",Some Like It Hot (1959),1959,"[romantic comedy, black and white, classic, Co..."
1285,[Comedy],Heathers (1989),1989,"[ADOLESCENCE IS HELL, high school, library, sn..."
1281,"[Comedy, Drama, War]","Great Dictator, The (1940)",1940,"[black and white, do kina, Chaplin, Nazis, amn..."
5060,"[Comedy, Drama, War]",M*A*S*H (a.k.a. MASH) (1970),1970,"[satire, Vietnam War, great soundtrack, Nation..."
922,"[Drama, Film-Noir, Romance]",Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),1950,"[AFI 100, Emerson must see, Hollywood, Billy W..."
3629,"[Adventure, Comedy, Romance]","Gold Rush, The (1925)",1925,"[black and white, mining, Alaska, amnesia, dan..."


We can examine the directions in feature space that the SVD picked out.  Here, we print the top ten tags associated with each dimension of the SVD.  We can see dimensions corresponding to classic movies, sci-fi franchises, comedies, and movies adapted from books.  (Note that the SVD has found that "based on a book", "adapted from:book", and "based on book" are all associated with each other.)

In [138]:
svd = tag_pipe.named_steps['svd']
vect = tag_pipe.named_steps['vectorizer']
[", ".join([vect.feature_names_[i] for i in c.argsort()[:-10:-1]])
 for c in svd.components_]

["Tumey's DVDs, imdb top 250, erlend's DVDs, classic, dvd, seen more than once, R, based on a book, National Film Registry",
 'R, ClearPlay, Nudity (Topless), movie to see, less than 300 ratings, Nudity (Topless - Brief), owned, based on a book, seen at the cinema',
 "seen more than once, 70mm, action, seen at the cinema, dvd, Futuristmovies.com, franchise, sci-fi, Eric's Dvds",
 'less than 300 ratings, based on a book, National Film Registry, adapted from:book, classic, 70mm, AFI 100, movie to see, AFI 100 (Cheers)',
 'less than 300 ratings, stylized, atmospheric, tense, 70mm, Criterion, quirky, humorous, seen more than once',
 'based on a book, adapted from:book, stylized, atmospheric, Criterion, disturbing, based on book, tense, Nudity (Topless)',
 'comedy, quirky, funny, humorous, seen more than once, satirical, classic, irreverent, witty',
 "Tumey's DVDs, less than 300 ratings, Bibliothek, based on a book, adapted from:book, movie to see, Eric's Dvds, funny, own",
 'movie to see, 

## Recommendation for a User

The models so far seem to do a pretty good job of suggesting movies similar to a particular movie.  Sometimes, this is all that is needed.  If we notice a user buying a particular movie, we could use this to suggest similar movies they might like.

Often, however, we wish to suggest a movie to a user.  One approach would be to calculate an "average movie" that a particular user enjoyed, and then use the nearest neighbors approach to find movies similar to that one.

To help us judge, we'll add the movie titles to the ratings table.

In [139]:
df_ratings_title = df_ratings.merge(df[['title']], left_on='movie_id', right_index=True)

And we'll look at user 9689, who seems to be a fan of horror.

In [140]:
uid = 9689
df_ratings_title[df_ratings_title.user_id == uid].sort_values('rating', ascending=False)

,movie_id,rating,user_id,title
1320721,2367,5.0,9689,King Kong (1976)
1320715,1982,5.0,9689,Halloween (1978)
1320709,1345,5.0,9689,Carrie (1976)
1320708,1339,5.0,9689,Dracula (Bram Stoker's Dracula) (1992)
1320706,1258,5.0,9689,"Shining, The (1980)"
1320705,1219,4.0,9689,Psycho (1960)
1320717,1994,4.0,9689,Poltergeist (1982)
1320713,1407,4.0,9689,Scream (1996)
1320718,2003,4.0,9689,Gremlins (1984)
1320716,1991,4.0,9689,Child's Play (1988)


We'll compute a weighted average movie, using the scores given as ratings.

In [141]:
m_locs = df_ratings[df_ratings.user_id == uid]['movie_id'].apply(lambda x: df.index.get_loc(x))
scaled_ratings = df_ratings[df_ratings.user_id == uid]['rating'] * 0.2
weighted_avg_movie = scaled_ratings.reshape(1,-1).dot(features[m_locs,:].toarray()) / len(scaled_ratings)

/home/ariel/miniconda2/envs/pydata/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  app.launch_new_instance()


And as expected, we get a bunch of horror movies.

In [142]:
dists, indices = nn.kneighbors(weighted_avg_movie)
df.iloc[indices[0]]

,categories,title,year,tag
id,,,,
4744,[Horror],Jeepers Creepers (2001),2001,"[horror, monster]"
3843,[Horror],Sleepaway Camp (1983),1983,"[monster, penis, tranny]"
2638,[Horror],"Mummy's Tomb, The (1942)",1942,"[Scary Movies To See on Halloween, mummy]"
2636,[Horror],"Mummy's Ghost, The (1944)",1944,"[Scary Movies To See on Halloween, mummy]"
2635,[Horror],"Mummy's Curse, The (1944)",1944,"[Scary Movies To See on Halloween, mummy]"
2783,[Horror],"Tomb of Ligeia, The (1965)",1965,"[Scary Movies To See on Halloween, Vincent Pri..."
2868,[Horror],Fright Night Part II (1989),1989,"[cheesy, vampire]"
2119,[Horror],Maximum Overdrive (1986),1986,[stephen king]
1349,[Horror],Nosferatu in Venice (a.k.a. Vampire in Venice)...,1986,"[Klaus Kinski, vampire]"


## Cooperative Learning

Content-based recommendation systems work well to find items similar to those a user already likes.  To help with that, we turn to a collaborative learning model.  Instead of finding movies similar to movies a user liked, we will find *users* similar to a given user, and then find the movies that they rated highly.

How do we describe users?  We could use demographic information, but we have better set of features handy: the users' ratings.  Two users are similar if they have rated movies similarly.

In [143]:
by_user_ratings = df_ratings.groupby('user_id').apply(
    lambda items: {i[1]: i[2] for i in items.itertuples()}) # 0 is index
features = DictVectorizer().fit_transform(by_user_ratings)

Previously, we have been using the default [Euclidean metric](https://en.wikipedia.org/wiki/Euclidean_distance) in the nearest neighbors calculation:

$$ d(x, y) =  \left| x - y \right|^2 \ . $$

For users, we will instead use the [cosine metric](https://en.wikipedia.org/wiki/Cosine_similarity),

$$ d(x, y) = 1 - \frac{ x\cdot y}{|x|\ |y|} \ , $$

which cares about angles between vectors in feature space.  This dependence on angle only lessens the effect of users using different scales to rate the movies

In [144]:
nn = NearestNeighbors(n_neighbors=20, metric='cosine', algorithm='brute').fit(features)

In [145]:
dists, indices = nn.kneighbors(features[by_user_ratings.index.get_loc(uid), :])
neighbors = [by_user_ratings.index[i] for i in indices[0]][1:]
ratings_grp = df_ratings_title[df_ratings_title['user_id'].isin(neighbors)] \
    .groupby('title')['rating']
len(ratings_grp)

367

There are a bunch of movies to consider, but we want to pick those that were most enjoyed by the neighbor users.  There's a few ways we might try.

### Attempt 1: Naive Mean

We could pick those that have the highest average rating, but these are typically movies reviewed by only one or two users.

In [146]:
ratings_grp.agg(['mean', 'count']).sort_values('mean', ascending=False)

,mean,count
title,,
Dracula (1931),5.000000,1
"Goonies, The (1985)",5.000000,1
Natural Born Killers (1994),5.000000,1
E.T. the Extra-Terrestrial (1982),5.000000,2
Rambling Rose (1991),5.000000,1
Man Bites Dog (C'est arrivé près de chez vous) (1992),5.000000,1
Runaway (1984),5.000000,1
Legend (1985),5.000000,1
Kiss the Girls (1997),5.000000,1


### Try 2: Naive Sum

Using the sum of the ratings will solve this particular problem, but will tend to highlight only those popular movies that almost everyone has seen.  There's no chance for a movie that's a hit with only some of the neighbors to shine through.

In [147]:
ratings_grp.agg(['sum', 'count']).sort_values('sum', ascending=False)

,sum,count
title,,
Jaws (1975),87.0,19
Alien (1979),81.0,18
"Shining, The (1980)",81.0,18
Psycho (1960),80.0,19
Carrie (1976),76.0,18
Poltergeist (1982),73.0,17
Scream (1996),69.0,17
Halloween (1978),64.0,15
"Exorcist, The (1973)",59.0,14


### Try 3: Bayesian Smoothing

One way to balance these two competing needs is to introduce a **Bayesian prior**.  This is the estimate we make that every movie has a rating $\mu$, before seeing any of the user ratings.  As we see ratings from users come it, we formulate a **posterior** estimate of the rating, taking into account both the prior and the new information provided by the ratings.  The posterior estimate of the rating, after seeing the $n$ reviews $\{x_i\}$, can be written as

$$ \frac{ \sum_{i=0}^n x_i + \mu N}{n + N} \ , $$

where $N$ reflects our level of confidence in the prior.  Adjusting $N$ affects the number of user reviews needed to push the posterior estimate away from $mu$.  Adjusting $mu$ affects where in the rankings movies with few reviews appear.  This technique is known as [Bayesian Smoothing](https://en.wikipedia.org/wiki/Additive_smoothing).

In this case, we use $\mu = 3$, $N = 5$, essentially starting each movie off with five reviews of three stars.  The top rated movies are all ones reviewed by our user.  Looking a little bit down the list, we see a recommendation for *The Thing*, which seems very reasonable.  *Ghostbusters* also ranks highly.  This is somewhat surprising, since our user didn't rate any comedies.  Evidently, other fans of horror films like *Ghostbusters* anyway, so maybe our user should give it a try.

In [148]:
def bayes_sum(N, mu):
    return lambda x: (x.sum() + mu*N) / (x.count() + N)

ratings_grp.aggregate(bayes_sum(5, 3)).sort_values(ascending=False)

title
Jaws (1975)                                     4.250000
Shining, The (1980)                             4.173913
Alien (1979)                                    4.173913
Poltergeist (1982)                              4.000000
Psycho (1960)                                   3.958333
Carrie (1976)                                   3.956522
Halloween (1978)                                3.950000
Nightmare on Elm Street, A (1984)               3.944444
Exorcist, The (1973)                            3.894737
Scream (1996)                                   3.818182
Gremlins (1984)                                 3.722222
Thing, The (1982)                               3.714286
Ghostbusters (a.k.a. Ghost Busters) (1984)      3.692308
Birds, The (1963)                               3.666667
Misery (1990)                                   3.636364
Omen, The (1976)                                3.625000
Blair Witch Project, The (1999)                 3.625000
American Werewolf in Lond

**Exercises:**

1. Explore the effect of different distance metrics on the recommendations.  The 'euclidean', 'manhattan', and 'cosine' metrics are some of the more common ones.  Check the `NearestNeighbors` documentation for additional options.

2. Add in the year as a feature of movies.  Should it be a continuous or categorical variable?  While it's likely that a user who likes movies from 1970 will also like movies from 1971 (suggesting a continuous feature), a user liking movies from 1970 and 1990 is no guarantee that they like movies from 1980 (suggesting a categorical feature).  Can you find an encoding that's in between these two options?  How does the scale of the year, if treated as a continuous feature, affect the KNN calculation?  You may want to use a `StandardScaler`, from `sklearn.preprocessing`, to reduce this effect.

3. In weighting reviews, we consider not reviewing a movie to be less of a recommendation that scoring it a one.  Shift the rating scale to change this, and see how the affects the resultant recommendations.

## Regression of Ratings

We have developed several ways to recommend movies to a user.  However, it is hard to tell how well these systems work, since most users have not rated most movies.  We may return a list of recommendations, none of which has been reviewed by the user.  Is that a sign the user doesn't like those movies, or that they just haven't seen the yet?

An alternative approach is develop a regression model to learn the ratings.  That is, given a user $u$ and an item $i$, we wish to build a model to predict $r_{ui}$, the rating given by $u$ to $i$.  In evaluating the performance of the model, we will consider only user/items pairs where a rating exists.  Our predictions, $\hat r_{ui}$ can be used to determine which unrated items may be preferred by a user.

In order to test how well we can predict rating that haven't been made, we train on only a portion of the data and then evaulate its perfomance on the remainder of the data.  We will take care to use a train-test split.  We will use the root mean-squared error (RMSE) to evaulate our model.  That is, we will compute

$$ \mbox{RMSE} = \sqrt{\frac{1}{n} \sum_{ui} \left(\hat r_{ui} - r_{ui}\right)^2} $$

where $r_{ui}$ are the ratings for $n$ $ui$-pairs in the test set.
  

For the purposes of having code execute quickly for demonstration, we will work with only a small portion of the users.

In [149]:
users = df_ratings['user_id'].unique()
np.random.seed(42)
user_sample = np.random.choice(users, 3000)
df_ratings_sample = df_ratings[df_ratings['user_id'].isin(user_sample)]
X_train, X_test, y_train, y_test = train_test_split(df_ratings_sample, df_ratings_sample['rating'])

## Baseline Model

We start with a baseline model that accounts for the fact that different users and different items will have different baseline scores.  This model has no interaction between the user and item:

$$ b_{ui} = \mu + b_u + b_i \ . $$

This may seem silly to worry about, but the baseline model is very important in practice.  The [winners of the Netflix Grand Prize](http://netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) succeeded largely on the strength of their baseline model.  Alone, it performed nearly as well as the existing Netflix model.

That model included a number of time-dependent terms, but we will consider each of the terms to be constant.  As such $\mu$ will be the average overall rating, $b_u$ will be the average difference from the global mean of all ratings by user $u$, and $b_i$ will be the average difference from the global mean of all ratings of item $i$.  If we one-hot encode both the users and movies, we can find these coefficients with a linear regression.

In [150]:
class Dictizer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.col].apply(lambda x: {x: 1})

In [151]:
user_dict = Pipeline([('dict', Dictizer('user_id')),
                      ('vect', DictVectorizer())])
movie_dict = Pipeline([('dict', Dictizer('movie_id')),
                       ('vect', DictVectorizer())])
union = FeatureUnion([('user_dict', user_dict),
                      ('movie_dict', movie_dict)])
lr_pipe = Pipeline([('union', union),
                    ('lr', Ridge(alpha=0))])

In [152]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('user_dict', Pipeline(steps=[('dict', Dictizer(col='user_id')), ('vect', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=True))])), ('movie_dict', Pipeline(steps=[('dict', Dictizer(col='movie_id')...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

A small function will help us calculate RMSE scores.

In [153]:
def rmse(model, X, y):
    return np.sqrt(mean_squared_error(model.predict(X), y))

To callibrate our expectations, the RMSE of the mean model is the square root of the variance.

In [154]:
np.sqrt(y_test.var())

1.0661338326706293

Our baseline model does about 18% better.

In [155]:
rmse(lr_pipe, X_test, y_test)

0.8798803883612244

However, our model scores significantly better on the training set.

In [156]:
rmse(lr_pipe, X_train, y_train)

0.84778024105566674

In [157]:
len(lr_pipe.named_steps['lr'].coef_)

11374

## Overfitting and Cross-validation

This discrepancy is a sign that we may be overfitting.  Training data is never perfectly clean.  It contains both the underlying signal, which you want to model, and some random noise, which you do not.  Since our model has over 11K parameters, it has plenty of flexibility to fit the noise as well as the signal.

The noise in this case is the same issue that prompted us to use a Bayesian mean to rate the movies suggested by neighbor users.  Prior to seeing any ratings for a item, for example, we would expect $b_i = 0$.  After seeing a single rating $r$, we shouldn't set $b_i = r - mu$.  Instead, we should update our estimate or $b_i$ somewhat in between these values, depending on our confidence in the prior.

While we could add in these prior ratings by hand, we can use regularization to produce the same effect.  Recall that in ridge regression, we add a term proportional to $\beta^2$ to the quantity being minimized:

$$ \left|y - X\cdot\beta\right|^2 + \alpha\ \left|\beta\right|^2 \ . $$

If the model is being used to take an average, as happens with one-hot encoding, the resultant coefficients will be

$$ \hat\beta_i = \frac{\sum_{j\mid X_{ij}=1} y_j}{n + \alpha} \ , $$

where $n$ is the number of $y_j$ included in the sum.  Note that this is equivalent to our previous Bayesian mean, with $\mu=0$ and $N=\alpha$.  We find the optimal value of $\alpha$ through cross-validation and a grid search.

In [158]:
gs = GridSearchCV(lr_pipe, {'lr__alpha': np.logspace(-1,1,5)}, n_jobs=-1)

`GridSearchCV` is itself an estimator.  When it is fit, it will run 3-fold cross validation (by default) on each set of hyperparameters, choose the best set of hyperparameters, and then train the estimator with the best hyperparameters on the full set of data.

In [159]:
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('user_dict', Pipeline(steps=[('dict', Dictizer(col='user_id')), ('vect', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=True))])), ('movie_dict', Pipeline(steps=[('dict', Dictizer(col='movie_id')...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'lr__alpha': array([  0.1    ,   0.31623,   1.     ,   3.16228,  10.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

We see that performance on the test set has improved somewhat.  (The predict method on the grid search object simply calls predict on the best fit estimator.)

In [160]:
rmse(gs, X_test, y_test)

0.87581242845382035

The best estimator, and its hyperparameter, are available as properties of the grid search object.  (Scikit Learn uses the convention that a trailing underscore indicates a property available only after a model has been fit.)

In [161]:
best_lr = gs.best_estimator_
gs.best_params_

{'lr__alpha': 3.1622776601683795}

In [162]:
predict_df = pd.DataFrame({'movie_id': X_train['movie_id'].unique(), 'user_id': 9689})
predict_df = predict_df.merge(df[['title']], left_on='movie_id', right_index=True)
predict_df['scores'] = best_lr.predict(predict_df)
predict_df.sort_values('scores', ascending=False).head(20)

,movie_id,user_id,title,scores
3450,1178,9689,Paths of Glory (1957),4.523738
970,858,9689,"Godfather, The (1972)",4.514612
416,318,9689,"Shawshank Redemption, The (1994)",4.507726
2296,5147,9689,Wild Strawberries (Smultronstället) (1957),4.500447
569,750,9689,Dr. Strangelove or: How I Learned to Stop Worr...,4.484474
2916,4298,9689,Rififi (Du rififi chez les hommes) (1955),4.471214
1831,2019,9689,Seven Samurai (Shichinin no samurai) (1954),4.463618
192,50,9689,"Usual Suspects, The (1995)",4.452710
5906,6918,9689,"Unvanquished, The (Aparajito) (1957)",4.452701
2339,7767,9689,"Best of Youth, The (La Meglio gioventù) (2003)",4.446025


**Exercises:**

1. Implement another user-item interaction model.  Here are a few ideas:
   - Measure how similar the user is to fans of the item, and regress against that distance.
   - Measure how similar the item is to items the user likes, and regress against that distance.
   - Use a nearest neighbors approach to report the scores given to an item by users similar to the user in question.
   
2. Blend two (or more) interaction models together.  These can be different algorithms from (1) or several matrix factorization algorithms with different hyperparameters.  Linear regression is a good way to automatically perform a weighted average together.  The BellKor team had success with **gradient-boosted trees** as blenders.

3. Add features of the movie or user to the blend.  A nonlinear model, such as **random forest** or gradient-boosted trees, may be able to detect that one of the component models does particularly well for a certain subset of features.

## References

- ["Recommendation Systems", from *Mining of Massive Datasets*](http://infolab.stanford.edu/~ullman/mmds/ch9.pdf), by Jure Leskovec, Anand Rajaraman, and Jeff Ullman
- ["Recommender Systems"](http://vikas.sindhwani.org/recommender.pdf), by Prem Melville and Vinkas Sindhwani
- [How to Build a Recommendation System](http://www.mickaellegal.com/blog/2014/1/30/how-to-build-a-recommender), by Mickaël Le Gal
- ["How to Build a Recommender System"](http://blogs.gartner.com/martin-kihn/how-to-build-a-recommender-system-in-python/), by Martin Kihn
- ["The BellKor Solution to the Netflix Grand Prize"](http://netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf), by Yehuda Koren
- ["Netflix Update: Try This at Home"](http://sifter.org/~simon/journal/20061211.html), by Simon Funk
- ["Alternating Least Squares Method for Collaborative Filtering"](http://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/), by Bugra Akyildiz

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*